# MS2 analysis

Jupyter notebook containing analyses for evaluating impact of MS2 and WPRE on barcode capture in nuclei datasets from Shin & Urbanek. Analyses correspond to !!!!!!! 

This notebook contains analyses related to quantifying and visualizing successful barcode capture with three different nuclear capturing strategies:
1) KASH only
2) KASH+MS2
3) KASH+MS2+WPRE

Input for this analysis requires:

1)
2)
3)

Output for this analysis includes:

Module and their versions used when generating figures for the paper can be found in 'requirements.txt', which is stored in our GitHub repository: !!!!!!

This code was last amended by Maddie Urbanek on !!!!!

## Notebook set-up

## Import datasets